In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from os import listdir
from os.path import isfile, join
import tqdm
import tensorflow as tf
from tensorflow import keras
import mysql.connector as msql
from mysql.connector import Error

### Data Processing

Processing data took hours, skip this if we already have the engineered_data file. Avoid running this section if the engineered_data.csv is already existed.

In [ ]:
# the datasets are too large, do not push to github!
# download the file on the desktop
mypath = "data"
files = [mypath+'/'+str(f) for f in listdir(mypath) if isfile(join(mypath, f))]


Create data dictionary and move these 5 files into it.

Output of listdir(mypath) should be:

data/members_v3.csv.7z 

data/sample_submission_zero.csv.7z

data/train.csv.7z

data/transactions.csv.7z

data/user_logs.csv.7z

In [ ]:
files

In [ ]:
if "unpacked_data" not in listdir('./'):
    os.makedirs("./unpacked_data")
    unpacked = False
else:
    print("Dictionary is already created")
    unpacked = True

In [6]:
# unpack data
import py7zr

if unpacked is False:
    extract_path = "unpacked_data"
    for file in tqdm.tqdm(files):
        with py7zr.SevenZipFile(file, mode='r') as z:
            data = z.extractall(extract_path)

In [4]:
#user_data = pd.read_csv("unpacked_data/user_logs.csv")
member_dat = pd.read_csv("unpacked_data/members_v3.csv")
transactions_dat = pd.read_csv('unpacked_data/transactions.csv')
train_dat = pd.read_csv('unpacked_data/train.csv')

In [5]:
'''
Functions that use to change the datatype according to columns' largest values for memory saving。

For integer:
For example, the maximum number that int8 can store is 127, and the minimum is - 128;
if the largest values for such column is smaller than the value，
we change the data type to int8
Perform this for all columns include int.

For float:
We change all features to float32
Source: https://www.kaggle.com/jeru666/did-you-think-of-these-features/notebook
'''

def change_datatype(df):
    int_cols = list(df.select_dtypes(include=["int64"]).columns)
    for col in int_cols:
        if ((np.max(df[col]) <= 127) and(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)
            
def change_datatype_float(df):
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)

In [6]:
transactions_dat.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0


The memory used before reducation

In [14]:
mem = transactions_dat.memory_usage(index=True).sum()
print("The memory usage before reduction is", mem /1024**2,"MB")

The memory usage before reduction is 513.7384204864502 MB


We change the datatype of transactions_dat for memory reduction.

In [15]:
change_datatype(transactions_dat)

The memory used after reduction

In [16]:
mem = transactions_dat.memory_usage(index=True).sum()
print("The memory usage before reduction is", mem /1024**2,"MB")

The memory usage before reduction is 513.7384204864502 MB


#### Feature Engineering with MySQL

In [22]:
try:
    engineered_df = pd.read_csv("engineered_data.csv")
    engineered_df = engineered_df.iloc[: , 1:]
except:
    pass

In [23]:
engineered_df.head(20)

,msno,n_transc,mean_plan_days,mean_list_price,mean_amount_paid,is_cancel,payment_method_id_x,payment_method_id_y,last_transaction_date,last_payment_days,last_list_price,last_payment,is_churn
0,///2pAPKetZe8zPqAwkYBjAUr+4pS8Rc6bsO4eGAlWI=,10,30.0000,99.0000,99.0000,0,41,1,20170210,30,99,99,0
1,///mTKUVQV0oYwb1acUes7piDWa9GLvLDDKru+THzQ0=,14,30.0000,124.0000,124.0000,1,41,1,20170206,30,99,99,0
2,//+uXIKe+NMJmGyARqgZzvpVF58WgsDauMtz+1WXOFQ=,16,30.0000,133.3750,133.3750,1,41,1,20170220,30,99,99,0
3,//0G+KopxUhyRVSoDHtzwjOe8Jv+2T/7cV7Ukz+x9FQ=,22,21.8182,108.3636,149.0000,0,34,1,20170228,30,149,149,0
4,//1tIUeF70BEHRxcQ5SuArPy7N8GVh4dBCaUutmysAQ=,8,30.0000,149.0000,149.0000,0,40,1,20170201,30,149,149,0
5,//1Ur/6FRY6wZWRd8tNJXkFTCb7b8ZAR6TTURi52gQ4=,26,28.8462,143.2692,149.0000,0,41,1,20170204,30,149,149,0
6,//1vaNpN6xyfTBfsNKR0j1e0Lv8/eRyQjPt+nKU1ehk=,22,30.0000,139.9091,139.9091,1,41,1,20170210,30,99,99,0
7,//2cvK2gfq1pTSyxgDdMwt0Q8CB5z/83FHrodGZMHoo=,22,21.8182,108.3636,149.0000,0,34,1,20170228,30,149,149,0
8,//3AMgquHVG7WXJ1YNXWPzFaHULFMsNW7IJ+Dsidupg=,3,52.3333,219.0000,219.0000,0,29,0,20161013,120,477,477,1
9,//3f/r1eOH9Qk3uqYYSs5e6qRMpUMezx2MAz5NDeTw0=,26,29.0000,143.2692,149.0000,0,37,1,20170207,30,149,149,0


In [96]:
import mysql.connector as msql
from mysql.connector import Error

def connect_sql(password, database = None):
    if database == None:
        conn = msql.connect(host='localhost', user='root',  
                        password=password)#give ur username, password
    else:
        conn = msql.connect(host='localhost', user='root',  
                        password=password, database = database)

        print("You're connected to database: ", database)
    return conn


passward = "Chen090718"

try:
    conn = connect_sql(passward)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE churn_prediction")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Error while connecting to MySQL 1007 (HY000): Can't create database 'churn_prediction'; database exists


Input transaction data into SQL

In [20]:
transactions_dat.dtypes

msno                      object
payment_method_id           int8
payment_plan_days          int16
plan_list_price            int16
actual_amount_paid         int16
is_auto_renew               int8
transaction_date           int32
membership_expire_date     int32
is_cancel                   int8
dtype: object

In [21]:
train_df = pd.merge(transactions_dat, train_dat, on = 'msno') # merge the datasets

In [22]:
# create the transactions table in sql server for later use

try:
    conn = connect_sql(passward, "churn_prediction")
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute('DROP TABLE IF EXISTS transactions;')
        sql = ('CREATE TABLE transactions(msno varchar(255),'+
                'payment_method_id int,payment_plan_days int,plan_list_price int, actual_amount_paid int,is_auto_renew int,'+
                'transaction_date int, membership_expire_date int, is_cancel int, is_churn int)')
        cursor.execute(sql)
        #loop through the data frame
        for i,row in tqdm.tqdm(train_df.iterrows()):
            #here %S means string values 
            sql = "INSERT INTO churn_prediction.transactions VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
except Error as e:
            print("Error while connecting to MySQL", e)

0it [00:00, ?it/s]

You're connected to database:  churn_prediction
Creating table....
Table is created....


15883148it [35:26:15, 124.50it/s]


In [23]:
sql_avg = '''SELECT 
                msno, 
                COUNT(payment_method_id) as n_transc, 
                AVG(payment_plan_days) as mean_plan_days, 
                AVG(plan_list_price) as mean_list_price, 
                AVG(actual_amount_paid) as mean_amount_paid,
                MAX(is_cancel)
            FROM churn_prediction.transactions 
            GROUP BY msno'''

In [103]:
conn = connect_sql(passward, "churn_prediction")
cursor = conn.cursor()
cursor.execute(sql_avg)
# Fetch all the records
result = cursor.fetchall()

You're connected to database:  churn_prediction


In [104]:
avg_df = pd.DataFrame(result, columns=['msno', 'n_transc', 'mean_plan_days','mean_list_price', 'mean_amount_paid', "is_cancel"])

In [105]:
sql_payment_mode = '''SELECT msno, payment_method_id, MAX(occurs)
                      FROM (SELECT msno,payment_method_id, count(*) as occurs
                            FROM churn_prediction.transactions
                            GROUP BY msno, payment_method_id
                      ) T1
                      GROUP BY msno'''

In [106]:
cursor.execute(sql_payment_mode)
# Fetch all the records
result = cursor.fetchall()

In [107]:
payment_df = pd.DataFrame(result, columns=['msno', 'payment_method_id', 'occurs']).drop("occurs", axis = 1)

In [108]:
sql_auto_mode = '''SELECT msno, is_auto_renew, MAX(occurs)
                      FROM (SELECT msno,is_auto_renew, count(*) as occurs
                            FROM churn_prediction.transactions
                            GROUP BY msno, is_auto_renew
                      ) T1
                      GROUP BY msno'''

In [109]:
cursor.execute(sql_auto_mode)
# Fetch all the records
result = cursor.fetchall()

In [110]:
auto_df = pd.DataFrame(result, columns=['msno', 'payment_method_id', 'occurs']).drop("occurs", axis = 1)

In [111]:
sql_date = '''SELECT o.msno, o.transaction_date, MAX(o.payment_plan_days), MAX(o.plan_list_price), MAX(o.actual_amount_paid), o.is_churn
              FROM churn_prediction.transactions o
                  LEFT JOIN churn_prediction.transactions b
                      ON o.msno = b.msno AND o.transaction_date <  b.transaction_date
              WHERE b.transaction_date is NULL
              GROUP BY o.msno
             '''

In [112]:
cursor.execute(sql_date)
# Fetch all the records
result = cursor.fetchall()

In [113]:
date_df = pd.DataFrame(result, columns=['msno', 'last_transaction_date', 'last_payment_days', 'last_list_price', 'last_payment', "is_churn"])

In [114]:
engineered_df = avg_df.merge(payment_df, on="msno").merge(auto_df, on="msno").merge(date_df, on="msno")

In [2]:
engineered_df.to_csv("engineered_data.csv")

NameError: name 'engineered_df' is not defined

## Logistic Regression Model

Using the engineered data

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [5]:
dat = engineered_df.iloc[:,2:].sample(frac = 1)
train, test = train_test_split(dat, test_size=0.2)

In [6]:
train_y = train['is_churn'].values
train_X = train.drop("is_churn", axis = 1).values
test_y = test['is_churn'].values
test_X = test.drop("is_churn", axis = 1).values

In [7]:
lr = LogisticRegression(random_state=0).fit(train_X, train_y)
lr.predict(test_X)
lr.score(test_X, test_y)

0.9358366861879176

### Random Forest

In [8]:
#import packages
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [9]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(train_X,train_y)
Y_pred=clf.predict(test_X)

In [10]:
print("Accuracy:",metrics.accuracy_score(test_y, Y_pred))

Accuracy: 0.9583910326456414


### Neural Network

In [11]:
model = keras.Sequential([

    keras.layers.Dense(11, activation='relu'),

    keras.layers.Dense(8, activation='relu'),
    
    keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
train_X = np.asarray(train_X).astype('float32')
train_y = np.asarray(train_y).astype('float32')

In [13]:
model.fit(train_X, train_y, epochs=2, batch_size=16)

Epoch 1/2
49647/49647 [==============================] - 31s 611us/step - loss: 1673575667940528339550208.0000 - accuracy: 0.9361
Epoch 2/2
49647/49647 [==============================] - 35s 698us/step - loss: 0.2375 - accuracy: 0.9361


In [14]:
test_X = np.asarray(test_X).astype('float32')
test_y = np.asarray(test_y).astype('float32')
test_loss, test_acc = model.evaluate(test_X, test_y, verbose=1)

6206/6206 [==============================] - 5s 750us/step - loss: 0.2383 - accuracy: 0.9358


### Summary

Among three different methods for this project, the random forest has the highest accuracy equals to 95.6 percent. The neural network and the logistic regression only have 93 percent accuracy.

In [25]:
correlation = engineered_df.iloc[: , 1:].corr()

In [28]:
correlation.style.background_gradient(cmap='coolwarm')

,n_transc,mean_plan_days,mean_list_price,mean_amount_paid,is_cancel,payment_method_id_x,payment_method_id_y,last_transaction_date,last_payment_days,last_list_price,last_payment,is_churn
n_transc,1.000000,-0.235657,-0.180184,-0.157400,0.359828,0.119564,0.259676,0.160268,-0.185836,-0.143945,-0.143086,-0.164563
mean_plan_days,-0.235657,1.000000,0.966967,0.959324,-0.058323,-0.226956,-0.293703,-0.678480,0.867903,0.838257,0.837591,0.233014
mean_list_price,-0.180184,0.966967,1.000000,0.995068,-0.026804,-0.285293,-0.353979,-0.672918,0.836446,0.870111,0.869190,0.239263
mean_amount_paid,-0.157400,0.959324,0.995068,1.000000,-0.036280,-0.308368,-0.349260,-0.672493,0.835087,0.872004,0.871620,0.233813
is_cancel,0.359828,-0.058323,-0.026804,-0.036280,1.000000,-0.012190,-0.007801,0.009730,-0.038708,-0.025540,-0.026124,0.149327
payment_method_id_x,0.119564,-0.226956,-0.285293,-0.308368,-0.012190,1.000000,0.528152,0.203678,-0.208521,-0.267508,-0.265399,-0.173518
payment_method_id_y,0.259676,-0.293703,-0.353979,-0.349260,-0.007801,0.528152,1.000000,0.284693,-0.258788,-0.322767,-0.321675,-0.299519
last_transaction_date,0.160268,-0.678480,-0.672918,-0.672493,0.009730,0.203678,0.284693,1.000000,-0.779012,-0.772198,-0.771737,-0.324870
last_payment_days,-0.185836,0.867903,0.836446,0.835087,-0.038708,-0.208521,-0.258788,-0.779012,1.000000,0.957807,0.957385,0.253733
last_list_price,-0.143945,0.838257,0.870111,0.872004,-0.025540,-0.267508,-0.322767,-0.772198,0.957807,1.000000,0.999278,0.258216


From the correlation table, we can see that the label we are trying to predict is not very correlated to features. Therefore, there is not a clear linear relationship, which might be the reason that the logistic regression do not have a high accuracy. 